## Interactive Map - CalEnviroScreen & Oil Wells

# Setting up the data

Started by importing pandas and geopandas.

In [ ]:
import pandas as pd
import geopandas as gpd

I then ask pandas to read the file and converted a few of the location variables to strings in order to allow for some of the sorting and to zfill the information.

In [ ]:
df = pd.read_csv('Data/Calenviroscreen4.csv',
    dtype=
    {
        'Census Tract':str,
        'California County':str,
        'Approximate Location': str
    }
)
df.head()

Above, we see that the leading zero is missing from the data, so I zfilled the information and then simplified the information to location and CalEnviroscreen percentile measures.

In [ ]:
df['Census Tract'] = df['Census Tract'].str.zfill(11)

In [ ]:
df.info()

In [ ]:
columns_to_keep = ['Census Tract',
                   'Total Population',
                   'California County',
                   'Approximate Location',
                   'CES 4.0 Score',
                   'CES 4.0 Percentile',
                   'CES 4.0 Percentile Range',
                   'PM2.5 Pctl',
                   'Groundwater Threats Pctl',
                   'Asthma Pctl',
                   'Low Birth Weight Pctl',
                   'Unemployment Pctl',
                   'Housing Burden Pctl',
                   'Pollution Burden Pctl',
                   'Drinking Water Pctl',
                   'Tox. Release Pctl',
                   'Cleanup Sites Pctl',
                   'Poverty Pctl',
                   'Housing Burden Pctl',
                   'Cardiovascular Disease Pctl']
df2 = df[columns_to_keep]
df2.info

Next, I narrow this information to the county of Los Angeles.

In [ ]:
df2LA = df2[df2['California County'] == 'Los Angeles']

And rename the Census Tracts to a FIPS variable in order to allow for analysis using one word later on. I use the info command after renaming to confirm that the data successfully narrowed from California as a whole and that the column name was successfully changed.

In [ ]:
df2LA.rename(columns = {'Census Tract':'FIPS'}, inplace = True) 

In [ ]:
df2LA.info()

Next, I want to merge this information with geometry data, in this case based on census tracts, because it is the measure used in CalEnviroscreen.

In [ ]:
tracts = gpd.read_file('Data/2020_Census_Tracts.geojson')
tracts.head()

In [ ]:
tracts['FIPS'] ='06' + '037' + tracts['CT20']

In [ ]:
CEStracts=tracts.merge(df2LA,on='FIPS')

After this, I also confirm that the projection of the geodataframe is mercator in order to allow us to join our calenviroscreen information with information about oil wells in the county.

In [ ]:
CEStracts.crs

# Oil Wells Data

Now, I am adding the oil wells data to the mix.

In [ ]:
dfoil = pd.read_csv('Data/Oil_Wells.csv')

I create a variable for active oil wells as well in case we add oil wells to any maps later in this file.

In [ ]:
df_act = dfoil.drop(dfoil[dfoil['WellStatus'] != 'A'].index)

In [ ]:
gdf_act = gpd.GeoDataFrame(df_act, 
                       crs='epsg:4326',
                       geometry=gpd.points_from_xy(df_act.Longitude, df_act.Latitude))

And now I join the oil data with CalEnviroscreen.

In [ ]:
# Perform a spatial join based on geographic coordinates
gdfoilCES = gpd.sjoin(gdf_act, CEStracts, how='right', op='within')
gdfoilCES.head()

And I use the info command to look at the data.

In [ ]:
gdfoilCES.info()

# How CES related to oil wells

In [ ]:
oilwellsbytract = gdfoilCES.FIPS.value_counts().rename_axis('FIPS').reset_index(name='oilwell_count')

In [ ]:
oilwellsbytract.info()

In [ ]:
# join the summary table back to the gdf
gdfoilCES=gdfoilCES.merge(oilwellsbytract,on='FIPS')

It doesn't make sense to adjust for population like we did in the lab as this feature of oil well count does not need to be adjusted by the number of people. So, next step is just to confirm that the merge worked, which it did.

In [ ]:
gdfoilCES.info()

In [ ]:
columnsfull = ['ShapeSTArea',
               'ShapeSTLength',
               'geometry',
                    'FIPS',
                  'Total Population',
                   'California County',
                   'Approximate Location',
                   'CES 4.0 Score',
                   'CES 4.0 Percentile',
                   'CES 4.0 Percentile Range',
                   'PM2.5 Pctl',
                   'Groundwater Threats Pctl',
                   'Asthma Pctl',
                   'Low Birth Weight Pctl',
                   'Unemployment Pctl',
                   'Housing Burden Pctl',
                   'Pollution Burden Pctl',
                   'Drinking Water Pctl',
                   'Tox. Release Pctl',
                   'Cleanup Sites Pctl',
                   'Poverty Pctl',
                   'Housing Burden Pctl',
                   'Cardiovascular Disease Pctl',
                    'oilwell_count']
gdfoilCES = gdfoilCES[columnsfull]
gdfoilCES.info()

In [ ]:
# Check for duplicated column names
duplicated_columns = gdfoilCES.columns[gdfoilCES.columns.duplicated()]
if len(duplicated_columns) > 0:
    print("Duplicated columns found:", duplicated_columns)

In [ ]:
# Drop duplicated columns
gdfoilCES = gdfoilCES.drop(columns=duplicated_columns)

In [ ]:
gdfoilCES.info()

## Interactive Map

This is a view zoomed into the area surrounding Inglewood.

In [ ]:
import folium

# add empty folium map
m = folium.Map(location=[33.9617, -118.3531],  # Inglewood, CA coordinates
               zoom_start=12,
               tiles='CartoDB positron', 
               attribution='CartoDB')

# plot choropleth over the base map
folium.Choropleth(
                  geo_data=gdfoilCES, # geo data
                  data=gdfoilCES, # data          
                  key_on='feature.properties.FIPS', # key, or merge column
                  columns=['FIPS','CES 4.0 Percentile'], # [key, value]
                  fill_color='RdPu',
                  nan_fill_color='white',  # set nan_fill_color to 'white' for NaN values
                  line_weight=0.1, 
                  fill_opacity=0.8,
                  line_opacity=0.2, # line opacity (of the border)
                  legend_name='CES 4.0 Percentile').add_to(m)

# create feature group
f2=folium.FeatureGroup(name='Oil Wells').add_to(m)

# add the wells to the feature group
for index, row in gdf_act.iterrows():
    folium.Circle(
        radius=1,
        color="black",
        location=[row.Latitude,row.Longitude],
        overlay=False).add_to(f2)

folium.LayerControl(position='topright',collapsed=True, autoZIndex=True).add_to(m)

m

This is a view zoomed into the area surrounding Beverly Hills.

In [ ]:
import folium

# add empty folium map
m = folium.Map(location=[34.0736, -118.4004],  # Beverly Hills, CA coordinates
               zoom_start=13,
               tiles='CartoDB positron', 
               attribution='CartoDB')

# plot choropleth over the base map
folium.Choropleth(
                  geo_data=gdfoilCES, # geo data
                  data=gdfoilCES, # data          
                  key_on='feature.properties.FIPS', # key, or merge column
                  columns=['FIPS','CES 4.0 Percentile'], # [key, value]
                  fill_color='Greens',
                  nan_fill_color='white',  # set nan_fill_color to 'white' for NaN values
                  line_weight=0.1, 
                  fill_opacity=0.8,
                  line_opacity=0.2, # line opacity (of the border)
                  legend_name='CES 4.0 Percentile').add_to(m)

# create feature group
f2=folium.FeatureGroup(name='Oil Wells').add_to(m)

# add the wells to the feature group
for index, row in gdf_act.iterrows():
    folium.Circle(
        radius=1,
        color="black",
        location=[row.Latitude,row.Longitude],
        overlay=False).add_to(f2)

folium.LayerControl(position='topright',collapsed=True, autoZIndex=True).add_to(m)

m